<a href="https://colab.research.google.com/github/SarahLares/MD_Data_exploratory/blob/master/preprocesamiento_bioanalisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import output
from datetime import datetime

# Definición de Funciones

## Liempieza de datos

In [35]:
# Pre procesamiento de la data de estudiantes de la escuela de nutrición
def nutricionPreProcesscing(base_dataframe):
  # Remove Bad data (data that is not possible to be decoded as a number)
  base_dataframe['notaf'].replace(to_replace= np.nan,value = 4, inplace=True)
  base_dataframe.fillna(0.0, axis=1, inplace=True)

  PI = base_dataframe.loc[base_dataframe['notad']=='PI'].index
  EE = base_dataframe.loc[base_dataframe['notad']=='EE'].index
  EI = base_dataframe.loc[base_dataframe['notad']=='EI'].index
  A = base_dataframe.loc[base_dataframe['notad']=='A'].index
  AP = base_dataframe.loc[base_dataframe['notad']=='AP'].index

  base_dataframe.drop(index=AP,inplace=True)
  base_dataframe.drop(index=A,inplace=True)
  base_dataframe.drop(index=PI,inplace=True)
  base_dataframe.drop(index=EE,inplace=True)
  base_dataframe.drop(index=EI,inplace=True)
  # Decode score values
  
  base_dataframe['notad'] = base_dataframe['notad'].replace('RT', np.nan).replace('NP', '0.0').astype(float)
  i = base_dataframe.loc[base_dataframe['equivalenciainterna']=='t'].index
  base_dataframe.drop(index=i,inplace=True)
  base_dataframe = base_dataframe.reset_index()
  
  return base_dataframe

## Decodificar período del curso

In [36]:
# Decodifica el periodo en el que se cursaron las materias
def decodeCoursePeriod(base_dataframe):
  data = []

  for w in base_dataframe['nombre.1']:
    data.append([w[0:4],w[5:7]])

  df = pd.DataFrame(data=data, columns=['ano_periodo','numero_periodo'])
  base_dataframe.drop(columns=['nombre.1'], inplace = True)
  df_1 = pd.concat( [base_dataframe, df], axis=1, join='inner' )
  return df_1

## Creación de Kardex

In [37]:
# Construye un nuevo Dataframe que refleje el record academico de cada estudiante (Kardex)
def buildStudentsRecords(base_dataframe):
  course_feature_tempplates = [
    '{}_times_taken',
    '{}_times_withdrawn',
    '{}_uc',
    '{}_avg',
    '{}_last_score',
    '{}_last_period_year',
    '{}_last_period_number'
  ]
  courses_codes = base_dataframe['codigo'].unique()
  courses_data_key = []
  for course_id in courses_codes:
    for feature_template in course_feature_tempplates:
      courses_data_key.append( feature_template.format(course_id) )
  
  statistics_features = [
    'avg',
    'weighted_avg',
    'uc_enrrolled',
    'uc_approved',
    'uc_withdrawn',
    'efficiency',
    'full_efficiency'
  ]

  students_ids = base_dataframe['estudiante_id'].unique()
  final_students_records = []

  # Used to display % of advancement only
  num_of_students = len(students_ids)
  students_proceced = 0
  advance = 0

  count = 0

  for student_id in students_ids:
    student_record = []
    
    # base statistics
    avg_sum = 0
    weighted_avg_sum = 0
    courses_taken = 0
    uc_enrrolled = 0
    uc_approved = 0
    uc_withdrawn = 0
    efficiency = 0

    for course_code in courses_codes:
      is_course = base_dataframe['codigo'] == course_code
      is_student = base_dataframe['estudiante_id'] == student_id
      student_course_info = base_dataframe.loc[(is_course & is_student)]

      if (student_course_info.empty):
        course_uc = base_dataframe.loc[is_course].iloc[0].get('uc')
        for feature_template in course_feature_tempplates:
          if (feature_template == '{}_uc'):
            student_record.append(course_uc)
          else:
            student_record.append( 0 )
      else:
        sorted_student_courses_info = student_course_info.sort_values(by=['ano_periodo', 'numero_periodo'])
        
        times_taken = sorted_student_courses_info.shape[0]
        times_withdrawn = sorted_student_courses_info['notad'].isna().sum()
        course_uc = sorted_student_courses_info.iloc[0].get('uc')
        course_avg = sorted_student_courses_info['notad'].mean()
        last_score = sorted_student_courses_info.iloc[-1].get('notad')

        #  times_taken
        student_record.append( times_taken )
        #  times_withdrawn
        student_record.append( times_withdrawn )
        #  uc
        student_record.append( course_uc )
        #  avg
        if (np.isnan(course_avg)):
          student_record.append( 0 )
        else:
          student_record.append( course_avg )
        #  last_score
        if (np.isnan(last_score)):
          student_record.append( 0 )
        else:
          student_record.append( last_score )
        #  last_period_year
        student_record.append( sorted_student_courses_info.iloc[-1].get('ano_periodo') )
        #  last_period_number
        student_record.append( sorted_student_courses_info.iloc[-1].get('numero_periodo') )

        if (not np.isnan(course_avg)):
          avg_sum += course_avg
          courses_taken += 1
          weighted_avg_sum += (course_uc * course_avg)

        uc_enrrolled += (course_uc * times_taken)
        if (not np.isnan(last_score) and last_score >= 10):
          uc_approved += course_uc
        uc_withdrawn += (course_uc * times_withdrawn)

    # Calc and add statistics
    # avg
    if (courses_taken < 1):
      student_record.append(0)
    else:
      student_record.append(avg_sum/courses_taken)
    # weighted_avg
    if ((uc_enrrolled - uc_withdrawn) < 1):
      student_record.append(0)
    else:
      student_record.append(weighted_avg_sum/(uc_enrrolled - uc_withdrawn))
    # uc_enrrolled
    student_record.append(uc_enrrolled)
    # uc_approved
    student_record.append(uc_approved)
    # uc_withdrawn
    student_record.append(uc_withdrawn)
    # efficiency
    if ((uc_enrrolled - uc_withdrawn) < 1):
      student_record.append(0)
    else:
      student_record.append(uc_approved/(uc_enrrolled - uc_withdrawn))
    # full_efficiency
    if (uc_enrrolled < 1):
      student_record.append(0)
    else:
      student_record.append(uc_approved/uc_enrrolled)
        
    final_students_records.append(student_record)
    # Used to display % of advancement only
    students_proceced = students_proceced + 1
    advance_new = (students_proceced/num_of_students)*100
    if ((advance_new - advance) > 0.9):
      advance = advance_new
      output.clear()
      print('{:.2f} %'.format(advance))
  return pd.DataFrame(data=final_students_records, columns=(courses_data_key + statistics_features), index=students_ids)

## Datos de los estudiantes

In [38]:
# Extrae la información de los estudiantes de los otros dos archivos
def info_estudiantes(base_dataframe,df_estudiantes,df_graduados,licenciatura): 

  df_2= pd.DataFrame(base_dataframe['estudiante_id'].unique(),
                     columns=['estudiante_id'])
  
  estudiantes = pd.merge(left=df_2,right=df_estudiantes, left_on='estudiante_id',
                         right_on='id', how='inner')
  
  estudiantes.drop(columns=['id'])
  graduados = pd.merge(left=df_2,right=df_graduados, left_on='estudiante_id',
                       right_on='id')
  
  graduados.drop(columns=['id'],inplace=True)
  estudiantes = pd.merge(left=estudiantes,right=graduados, left_on='estudiante_id',
                         right_on='estudiante_id', how ='outer')
  estudiantes.drop(columns=['id'],inplace=True)
  estudiantes['Fecha'] = estudiantes['Fecha'].replace(np.nan, 0)
  estudiantes['graduado'] =  np.where(estudiantes['Fecha']==0, False, True)
  estudiantes['Fecha'] = estudiantes['Fecha'].replace(0, '2017-01-01')
  estudiantes['MENCION'] = estudiantes['MENCION'].replace(np.nan, False)
  estudiantes['MENCION'] = estudiantes['MENCION'].replace(licenciatura, True,regex=True)
  estudiantes['MENCION'] = estudiantes['MENCION'].replace(r'\w', False,regex=True )
  estudiantes.rename(columns={'MENCION':'graduado_nutricion'}, inplace=True)
  estudiantes['fechanac'] = pd.to_datetime(estudiantes['fechanac'])
  
  estudiantes['Edad_actual'] = estudiantes['fechanac'].apply(lambda x : 2017 - x.year)
  estudiantes['Edad_ingreso'] = estudiantes['Edad_actual'] - estudiantes['anoing'].apply(lambda x : 2017 - int(x))
  estudiantes['Fecha'] = pd.to_datetime(estudiantes['Fecha'])
  estudiantes['nro_de años_universidad'] = estudiantes['Fecha'].apply(lambda x : x.year) -  estudiantes['anoing']
  estudiantes.drop(columns=['Fecha','tipodi_id','fechanac'], inplace=True)
  estudiantes.set_index('estudiante_id', inplace=True) 
  return estudiantes

# Importación de datos

In [39]:
url_graduados = 'https://raw.githubusercontent.com/SarahLares/MD_Data_exploratory/master/graduados.csv'
url_estudiantes = 'https://raw.githubusercontent.com/SarahLares/MD_Data_exploratory/master/estudiantes.csv'
url_b = 'https://raw.githubusercontent.com/SarahLares/MD_Data_exploratory/master/carrera_52.csv'

In [40]:
df_estudiantes = pd.read_csv(url_estudiantes, delimiter=';')
print(f'** null values: \n{df_estudiantes.isnull().sum()}')
print(f'** shape: {df_estudiantes.shape}')

** null values: 
id           0
letradi      0
tipodi_id    0
anoing       0
fechanac     0
sexo         0
dtype: int64
** shape: (201375, 6)


In [41]:
df_graduados = pd.read_csv(url_graduados,delimiter=';')
df_estudiantes = pd.read_csv(url_estudiantes,delimiter=';')

In [42]:
df_bioanalisis = pd.read_csv(url_b,delimiter=';')
df_bioanalisis.isnull().sum()

estudiante_id           0
codigo                  0
nombre                  0
uc                     26
notaf                   1
notad                   0
tipoexamen              0
seccion                 0
equivalenciainterna     0
nombre.1                0
ano                     0
nombre.2                0
dtype: int64

In [43]:
df_bioanalisis.head()

,estudiante_id,codigo,nombre,uc,notaf,notad,tipoexamen,seccion,equivalenciainterna,nombre.1,ano,nombre.2
0,1116,0902003281,BIOQUIMICA CLINICA II,5.0,AP,00,F,A,f,2010-01S,2010,LICENCIADO EN BIOANÁLISIS
1,35654,0902003277,BIOQUIMICA CLINICA I,5.0,10,10,F,A,f,2010-02S,2010,LICENCIADO EN BIOANÁLISIS
2,1116,0902003281,BIOQUIMICA CLINICA II,5.0,AP,00,F,A,f,2010-02S,2010,LICENCIADO EN BIOANÁLISIS
3,58302,0902003605,APRENDER A APRENDER,2.0,13,13,F,A,f,2010-02S,2010,LICENCIADO EN BIOANÁLISIS
4,1591,0902003282,HEMATOLOGIA II,5.0,AP,11,D,A,f,2010-02S,2010,LICENCIADO EN BIOANÁLISIS


# Pre Procesamiento

In [44]:
a = pd.to_datetime(df_graduados['Fecha'])

In [45]:
clean_df_bioanalisis= nutricionPreProcesscing(df_bioanalisis)
clean_df_bioanalisis = decodeCoursePeriod(clean_df_bioanalisis)
bioanalisis_students_records = buildStudentsRecords(clean_df_bioanalisis)

99.29 %


In [46]:
bioanalisis_students_records.head()

,0902003281_times_taken,0902003281_times_withdrawn,0902003281_uc,0902003281_avg,0902003281_last_score,0902003281_last_period_year,0902003281_last_period_number,0902003277_times_taken,0902003277_times_withdrawn,0902003277_uc,0902003277_avg,0902003277_last_score,0902003277_last_period_year,0902003277_last_period_number,0902003605_times_taken,0902003605_times_withdrawn,0902003605_uc,0902003605_avg,0902003605_last_score,0902003605_last_period_year,0902003605_last_period_number,0902003282_times_taken,0902003282_times_withdrawn,0902003282_uc,0902003282_avg,0902003282_last_score,0902003282_last_period_year,0902003282_last_period_number,0902003465_times_taken,0902003465_times_withdrawn,0902003465_uc,0902003465_avg,0902003465_last_score,0902003465_last_period_year,0902003465_last_period_number,0902003280_times_taken,0902003280_times_withdrawn,0902003280_uc,0902003280_avg,0902003280_last_score,...,0902003606_uc,0902003606_avg,0902003606_last_score,0902003606_last_period_year,0902003606_last_period_number,3508_times_taken,3508_times_withdrawn,3508_uc,3508_avg,3508_last_score,3508_last_period_year,3508_last_period_number,00090200253_times_taken,00090200253_times_withdrawn,00090200253_uc,00090200253_avg,00090200253_last_score,00090200253_last_period_year,00090200253_last_period_number,Estandar_times_taken,Estandar_times_withdrawn,Estandar_uc,Estandar_avg,Estandar_last_score,Estandar_last_period_year,Estandar_last_period_number,0902003480_times_taken,0902003480_times_withdrawn,0902003480_uc,0902003480_avg,0902003480_last_score,0902003480_last_period_year,0902003480_last_period_number,avg,weighted_avg,uc_enrrolled,uc_approved,uc_withdrawn,efficiency,full_efficiency
1116,5,0,5.0,5.6,11.0,2012,01,0,0,5.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,...,2.0,0.0,0.0,0,0,0,0,30.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,13.720000,10.861538,65.0,45.0,0.0,0.692308,0.692308
35654,1,0,5.0,10.0,10.0,2011,02,1,0,5.0,10.0,10.0,2010,02,1,0,2.0,13.0,13.0,2011,01,1,0,5.0,14.0,14.0,2011,02,1,0,3.0,15.0,15.0,2010,01,1,0,5.0,13.0,13.0,...,2.0,0.0,0.0,0,0,0,0,30.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,13.368421,14.608696,92.0,92.0,0.0,1.000000,1.000000
58302,1,0,5.0,9.0,9.0,2011,02,1,0,5.0,10.0,10.0,2010,02,1,0,2.0,13.0,13.0,2010,02,1,0,5.0,12.0,12.0,2011,02,1,0,3.0,14.0,14.0,2010,01,1,0,5.0,11.0,11.0,...,2.0,0.0,0.0,0,0,0,0,30.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,13.000000,13.598039,102.0,92.0,0.0,0.901961,0.901961
1591,1,0,5.0,11.0,11.0,2011,01,2,0,5.0,6.0,10.0,2010,02,0,0,2.0,0.0,0.0,0,0,1,0,5.0,11.0,11.0,2010,02,0,0,3.0,0.0,0.0,0,0,1,0,5.0,12.0,12.0,...,2.0,0.0,0.0,0,0,0,0,30.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,10.750000,13.353846,65.0,58.0,0.0,0.892308,0.892308
52770,1,0,5.0,10.0,10.0,2011,02,1,0,5.0,10.0,10.0,2010,02,0,0,2.0,0.0,0.0,0,0,1,0,5.0,13.0,13.0,2011,02,1,0,3.0,13.0,13.0,2010,01,1,0,5.0,11.0,11.0,...,2.0,0.0,0.0,0,0,0,0,30.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,12.937500,14.037037,110.0,106.0,2.0,0.981481,0.963636


In [47]:
df_students_info =info_estudiantes(clean_df_bioanalisis,df_estudiantes,df_graduados,'LICENCIADO EN BIOAN')

In [48]:
final_bioanalisis_students_records = bioanalisis_students_records.join(df_students_info)

In [49]:
final_bioanalisis_students_records.head()

,0902003281_times_taken,0902003281_times_withdrawn,0902003281_uc,0902003281_avg,0902003281_last_score,0902003281_last_period_year,0902003281_last_period_number,0902003277_times_taken,0902003277_times_withdrawn,0902003277_uc,0902003277_avg,0902003277_last_score,0902003277_last_period_year,0902003277_last_period_number,0902003605_times_taken,0902003605_times_withdrawn,0902003605_uc,0902003605_avg,0902003605_last_score,0902003605_last_period_year,0902003605_last_period_number,0902003282_times_taken,0902003282_times_withdrawn,0902003282_uc,0902003282_avg,0902003282_last_score,0902003282_last_period_year,0902003282_last_period_number,0902003465_times_taken,0902003465_times_withdrawn,0902003465_uc,0902003465_avg,0902003465_last_score,0902003465_last_period_year,0902003465_last_period_number,0902003280_times_taken,0902003280_times_withdrawn,0902003280_uc,0902003280_avg,0902003280_last_score,...,3508_avg,3508_last_score,3508_last_period_year,3508_last_period_number,00090200253_times_taken,00090200253_times_withdrawn,00090200253_uc,00090200253_avg,00090200253_last_score,00090200253_last_period_year,00090200253_last_period_number,Estandar_times_taken,Estandar_times_withdrawn,Estandar_uc,Estandar_avg,Estandar_last_score,Estandar_last_period_year,Estandar_last_period_number,0902003480_times_taken,0902003480_times_withdrawn,0902003480_uc,0902003480_avg,0902003480_last_score,0902003480_last_period_year,0902003480_last_period_number,avg,weighted_avg,uc_enrrolled,uc_approved,uc_withdrawn,efficiency,full_efficiency,letradi,anoing,sexo,graduado_nutricion,graduado,Edad_actual,Edad_ingreso,nro_de años_universidad
1116,5,0,5.0,5.6,11.0,2012,01,0,0,5.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,...,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,13.720000,10.861538,65.0,45.0,0.0,0.692308,0.692308,V,2000,M,True,True,34,17,14
1380,0,0,5.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,...,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,18.000000,18.000000,30.0,30.0,0.0,1.000000,1.000000,V,2003,M,True,True,33,19,7
1473,1,0,5.0,10.0,10.0,2010,01,0,0,5.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,3.0,0.0,0.0,0,0,0,0,5.0,0.0,0.0,...,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,11.000000,16.216216,37.0,35.0,0.0,0.945946,0.945946,V,2002,F,True,True,34,19,9
1526,2,0,5.0,7.5,10.0,2010,02,0,0,5.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,1,0,5.0,10.0,10.0,2010,02,0,0,3.0,0.0,0.0,0,0,1,0,5.0,10.0,10.0,...,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,9.861111,10.617371,71.0,52.0,0.0,0.732394,0.732394,V,2003,F,True,True,34,20,9
1591,1,0,5.0,11.0,11.0,2011,01,2,0,5.0,6.0,10.0,2010,02,0,0,2.0,0.0,0.0,0,0,1,0,5.0,11.0,11.0,2010,02,0,0,3.0,0.0,0.0,0,0,1,0,5.0,12.0,12.0,...,0.0,0.0,0,0,0,0,5.0,0.0,0.0,0,0,0,0,4.0,0.0,0.0,0,0,0,0,2.0,0.0,0.0,0,0,10.750000,13.353846,65.0,58.0,0.0,0.892308,0.892308,V,2001,M,True,True,32,16,11


In [50]:
from google.colab import files
final_bioanalisis_students_records.to_csv('data_preprocesada_bioanalisis.csv')
files.download('data_preprocesada_bioanalisis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>